In [2]:
import os
from glob import glob

import numpy as np
import pandas as pd
import xarray as xr

%matplotlib widget
import matplotlib.pyplot as plt

from SCDA_v2 import SCDA_v20

In [3]:
location = "greenland"
cloud_df = pd.read_parquet(f"../data/caliop-slstr-{location}-maxsza80.parquet")

def clean_cloud(df):
    return df[df.solar_zenith_angle < 80]

cloud_df = clean_cloud(cloud_df)

In [4]:
#SCDA_v20(R550, R16, BT37, BT11, BT12)

slstr_cloud, ndsi = SCDA_v20(cloud_df['S1'].values, cloud_df['S5'].values, cloud_df['S7'].values, cloud_df['S8'].values, cloud_df['S9'].values)


In [5]:
caliop_cloud = cloud_df['Number_Layers_Found'] > 0            
    
    
good_detected_cloud = np.sum(slstr_cloud & caliop_cloud)
wrong_detected_cloud = np.sum(slstr_cloud & ~caliop_cloud)
            
good_detected_clear = np.sum(~slstr_cloud & ~caliop_cloud)
wrong_detected_clear = np.sum(~slstr_cloud & caliop_cloud)
            
is_cloud = np.sum(caliop_cloud)
is_not_cloud = np.sum(~caliop_cloud)
    
df = pd.DataFrame({'TPR': good_detected_cloud / is_cloud, 
                     'FPR': wrong_detected_cloud / is_not_cloud,
                     'TPR_clear': good_detected_clear / is_not_cloud,
                     'FPR_clear': wrong_detected_clear / is_cloud}, index=[0])
df

,TPR,FPR,TPR_clear,FPR_clear
0,0.81991,0.132132,0.867868,0.18009
